<a href="https://colab.research.google.com/github/hurratanvir/Coursera_Capstone/blob/master/Capstone_cluster_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim 
import geocoder

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
print("Libraries imported.")

Libraries imported.


In [6]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')
neighbour_list =[]
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighbour_list.append(row.text)
KL_df = pd.DataFrame({'neighbourhood': neighbour_list})
KL_df

,neighbourhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan
5,Bandar Tun Razak
6,Bangsar
7,Bangsar Park
8,Bangsar South
9,Batu 11 Cheras


In [8]:
KL_df['neighbourhood']

0                       Alam Damai
1             Ampang, Kuala Lumpur
2                 Bandar Menjalara
3            Bandar Sri Permaisuri
4             Bandar Tasik Selatan
5                 Bandar Tun Razak
6                          Bangsar
7                     Bangsar Park
8                    Bangsar South
9                   Batu 11 Cheras
10              Batu, Kuala Lumpur
11                     Brickfields
12                   Bukit Bintang
13                     Bukit Jalil
14                     Bukit Kiara
15                     Bukit Nanas
16                  Bukit Petaling
17                     Bukit Tunku
18            Cheras, Kuala Lumpur
19                        Chow Kit
20               Damansara Heights
21           Damansara Town Centre
22         Damansara, Kuala Lumpur
23                      Dang Wangi
24                   Desa Petaling
25      Federal Hill, Kuala Lumpur
26                    Happy Garden
27    Jalan Cochrane, Kuala Lumpur
28                  

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in KL_df["neighbourhood"].tolist() ]
coords

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1511418740671178, 101.69937468268016],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.136442497863743, 101.69029641203808],
 [3.1419057131529953, 101.67967833034047],
 [3.1566851613075664, 101.6

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
KL_df['Latitude'] = df_coords['Latitude']
KL_df['Longitude'] = df_coords['Longitude']
KL_df

,neighbourhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


In [11]:
KL_df.to_csv("kl_df.csv", index=False)
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [12]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(KL_df['Latitude'], KL_df['Longitude'], KL_df['neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:
map_kl.save('map_kl.html')

In [16]:
CLIENT_ID = 'A4QS3WE0QWXS01XZW0MW0QYY2JEJCP0X1ULH5U3AEHYP5I55' # your Foursquare ID
CLIENT_SECRET = 'DDN31D3YAZDT2TH5X111UF5IBTXFJGYMTX0KQ1NLPARC2M2D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A4QS3WE0QWXS01XZW0MW0QYY2JEJCP0X1ULH5U3AEHYP5I55
CLIENT_SECRET:DDN31D3YAZDT2TH5X111UF5IBTXFJGYMTX0KQ1NLPARC2M2D


In [17]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(KL_df['Latitude'], KL_df['Longitude'], KL_df['neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]["groups"][0]["items"]
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
venues

[('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  'Pengedar Shaklee Kuala Lumpur',
  3.0612348,
  101.74069559999998,
  'Supplement Shop'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  'Jc Deli 皆喜食坊',
  3.0583973,
  101.74856,
  'Food & Drink Shop'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  'Machi Noodle 妈子面',
  3.0576951281341227,
  101.74663516803412,
  'Noodle House'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  'Minang Tomyam',
  3.0571853995475733,
  101.74981152527421,
  'Seafood Restaurant'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  '628火焰鑫茶室',
  3.058442456011591,
  101.74794730881473,
  'Chinese Restaurant'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  '沙巴生肉面',
  3.0577150980227055,
  101.74909624565971,
  'Chinese Restaurant'),
 ('Alam Damai',
  3.0576900000000364,
  101.74388000000005,
  'Restoran Ikbal',
  3.0611337960965446,
  101.75021961655828,
  'Restaurant'),
 ('A

In [18]:
venues_df = pd.DataFrame(venues)

# define the column names


print(venues_df.shape)
venues_df.head()

(7084, 7)


,0,1,2,3,4,5,6
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


In [19]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7084, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


In [29]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhood"],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()
 #sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

ValueError: ignored

In [0]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]
kl_merged.loc[kl_merged['Cluster Labels'] == 1]
kl_merged.loc[kl_merged['Cluster Labels'] == 2]
# as per the obervation , there are highest number of cells or neighbiurs in central area of the Kuala Lumpur area. hence thats the best place to buy the plot for setting up the shopping mall .